<a href="https://colab.research.google.com/github/SergeiDS/Test_for_pyspark/blob/main/collab_pyspark_test_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Установка через pip

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 44 kB/s 
     |████████████████████████████████| 199 kB 43.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=f6b9801cf9d84e35c144f48ef38e2683f80c61123d66134ebb61cb088de5dfca
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [ ]:
spark = SparkSession.builder.appName("Test").getOrCreate()

In [ ]:
spark.version

'3.3.0'

https://www.kaggle.com/AnalyzeBoston/crimes-in-boston  данные отсюда

In [ ]:
from google.colab import files
files.upload()

Saving crime.csv to crime.csv
Saving offense_codes.csv to offense_codes.csv


In [ ]:
codes = spark.read.csv('offense_codes.csv', header = True)
crime = spark.read.csv('crime.csv', header = True)

In [ ]:
codes.show(10)

+----+--------------------+
|CODE|                NAME|
+----+--------------------+
| 612|LARCENY PURSE SNA...|
| 613| LARCENY SHOPLIFTING|
| 615|LARCENY THEFT OF ...|
|1731|              INCEST|
|3111|LICENSE PREMISE V...|
|2646|LIQUOR - DRINKING...|
|2204|LIQUOR LAW VIOLATION|
|3810|M/V ACCIDENT - IN...|
|3801|M/V ACCIDENT - OTHER|
|3807|M/V ACCIDENT - OT...|
+----+--------------------+
only showing top 10 rows



Сразу создадим новый столбец в таблице codes, т.к. он будет нужен для определения самых частов типов преступлений, для этого создадим UDF функцию и при помощи обычной питоновской функции оставим необходимую часть строки до разделителя "-"

In [ ]:
def st(x):
    return x.partition(' -')[0]
convertUDF = F.udf(lambda z: st(z))

In [ ]:
codes = codes.withColumn("crime_type", convertUDF(F.col("NAME")))

In [ ]:
codes.show(10)

+----+--------------------+--------------------+
|CODE|                NAME|          crime_type|
+----+--------------------+--------------------+
| 612|LARCENY PURSE SNA...|LARCENY PURSE SNATCH|
| 613| LARCENY SHOPLIFTING| LARCENY SHOPLIFTING|
| 615|LARCENY THEFT OF ...|LARCENY THEFT OF ...|
|1731|              INCEST|              INCEST|
|3111|LICENSE PREMISE V...|LICENSE PREMISE V...|
|2646|LIQUOR - DRINKING...|              LIQUOR|
|2204|LIQUOR LAW VIOLATION|LIQUOR LAW VIOLATION|
|3810|M/V ACCIDENT - IN...|        M/V ACCIDENT|
|3801|M/V ACCIDENT - OTHER|        M/V ACCIDENT|
|3807|M/V ACCIDENT - OT...|        M/V ACCIDENT|
+----+--------------------+--------------------+
only showing top 10 rows



In [ ]:
crime.show(10)

+---------------+------------+--------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------------+-----------+------------+--------------------+
|INCIDENT_NUMBER|OFFENSE_CODE|  OFFENSE_CODE_GROUP| OFFENSE_DESCRIPTION|DISTRICT|REPORTING_AREA|SHOOTING|   OCCURRED_ON_DATE|YEAR|MONTH|DAY_OF_WEEK|HOUR|  UCR_PART|           STREET|        Lat|        Long|            Location|
+---------------+------------+--------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------------+-----------+------------+--------------------+
|     I182070945|       00619|             Larceny|  LARCENY ALL OTHERS|     D14|           808|    null|2018-09-02 13:00:00|2018|    9|     Sunday|  13|  Part One|       LINCOLN ST|42.35779134|-71.13937053|(42.35779134, -71...|
|     I182070943|       01402|           Vandalism|           VANDALISM|     C11|   

In [ ]:
crime.printSchema()

root
 |-- INCIDENT_NUMBER: string (nullable = true)
 |-- OFFENSE_CODE: string (nullable = true)
 |-- OFFENSE_CODE_GROUP: string (nullable = true)
 |-- OFFENSE_DESCRIPTION: string (nullable = true)
 |-- DISTRICT: string (nullable = true)
 |-- REPORTING_AREA: string (nullable = true)
 |-- SHOOTING: string (nullable = true)
 |-- OCCURRED_ON_DATE: string (nullable = true)
 |-- YEAR: string (nullable = true)
 |-- MONTH: string (nullable = true)
 |-- DAY_OF_WEEK: string (nullable = true)
 |-- HOUR: string (nullable = true)
 |-- UCR_PART: string (nullable = true)
 |-- STREET: string (nullable = true)
 |-- Lat: string (nullable = true)
 |-- Long: string (nullable = true)
 |-- Location: string (nullable = true)



обратим внимание, что Lat и Long строки, переведем их в формат float, т.к. мы будем считать их среднее значение

In [ ]:
crime=crime.withColumn('Lat',crime['Lat'].cast("float").alias('Lat'))
crime=crime.withColumn('Long',crime['Long'].cast("float").alias('Long'))

In [ ]:
crime.select("DISTRICT").distinct().count()

13

In [ ]:
crime.select("DISTRICT").distinct().show()

+--------+
|DISTRICT|
+--------+
|      C6|
|    null|
|      B2|
|     C11|
|     E13|
|      B3|
|      E5|
|     A15|
|      A7|
|     D14|
|      D4|
|     E18|
|      A1|
+--------+



Заметим, что уникальных значений для районов 13, но есть null, т.к. в дальнейшнем по заданию нужна будет группировка именно по районам, уберем эти значения из выборки

In [ ]:
crime = crime.filter("DISTRICT is not NULL") 

In [ ]:
crime.select("DISTRICT").distinct().count()

12

In [ ]:
crime.printSchema()

root
 |-- INCIDENT_NUMBER: string (nullable = true)
 |-- OFFENSE_CODE: string (nullable = true)
 |-- OFFENSE_CODE_GROUP: string (nullable = true)
 |-- OFFENSE_DESCRIPTION: string (nullable = true)
 |-- DISTRICT: string (nullable = true)
 |-- REPORTING_AREA: string (nullable = true)
 |-- SHOOTING: string (nullable = true)
 |-- OCCURRED_ON_DATE: string (nullable = true)
 |-- YEAR: string (nullable = true)
 |-- MONTH: string (nullable = true)
 |-- DAY_OF_WEEK: string (nullable = true)
 |-- HOUR: string (nullable = true)
 |-- UCR_PART: string (nullable = true)
 |-- STREET: string (nullable = true)
 |-- Lat: float (nullable = true)
 |-- Long: float (nullable = true)
 |-- Location: string (nullable = true)



In [ ]:
codes.printSchema()

root
 |-- CODE: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- crime_type: string (nullable = true)



ЗАДАНИЕ

In [ ]:
crime.select('INCIDENT_NUMBER').distinct().count()

280934

Проанализировав данные в таблице, заметим, что каждое преступление характеризуется его входящим номером(INCIDENT_NUMBER), но под одним номером может быть несколько типов преступлений(OFFENSE_CODE), поэтому в дальнейшем при подсчете количества преступлений будем брать уникальные значения INCIDENT_NUMBER.


"Теоретически существует вариант, что каждое преступление следует считать отдельно, т.е. если у нескольких типов преступлений один номер, то считать каждое отдельно, но т.к. у каждого типа будет одни и те же координаты, было принято решение использовать уникальное значение." В реальной задаче стоило бы уточнить ТЗ =)

Для простоты объединим среднее значение широты/долготы по району с общим числом преступлений в районе в одну табличку(total_crime). Группируем по номеру происшествия.
По заданию найти:

●crimes_total - общее количество преступлений в этом районе

● lat - широта координаты района, расчитанная как среднее по всем широтам
инцидентов

● lng - долгота координаты района, расчитанная как среднее по всем долготам
инцидентов


In [ ]:
total_crime = (
    crime
    .groupBy("DISTRICT")
    .agg(
        F.approx_count_distinct('INCIDENT_NUMBER').alias("crimes_total"),
         F.avg('Lat').alias("lat"),
        F.avg('long').alias("lng") 
    ))
total_crime.show()

+--------+------------+------------------+------------------+
|DISTRICT|crimes_total|               lat|               lng|
+--------+------------+------------------+------------------+
|      C6|       20337|  42.2121224773585|-70.85561028359758|
|      B2|       41788|  42.3160037118871|-71.07569935580469|
|     C11|       39282| 42.29263737128488|-71.05125984476538|
|     E13|       16582| 42.30980363405438| -71.0980047618084|
|      B3|       32197|42.283059397131744|-71.07894938323396|
|      E5|       11669| 42.19796996617494|-71.00440886354875|
|     A15|        5779| 42.17915540166574|-70.74472495377354|
|      A7|       11839| 42.36070265992561|-71.00394839443797|
|     D14|       18879| 42.34350730061144| -71.1312545647069|
|      D4|       33963|42.341242479947674|-71.07725037365037|
|     E18|       16385| 42.26268049226358|-71.11891999428089|
|      A1|       30165| 42.33123074202031| -71.0199188480901|
+--------+------------+------------------+------------------+



● crimes_monthly - медиана числа преступлений в месяц в этом районе. Немного непонятно указано в тз, но будем считать медиану т.е. для каждого района будем считать медиану по месяцам и годам за всю историю наблюдений

In [ ]:
temp_for_median = (
    crime
    .groupBy("DISTRICT", "YEAR", "MONTH")
    .agg(
        F.approx_count_distinct('INCIDENT_NUMBER').alias("crimes_each_month"),
    ))

In [ ]:
crime_median = (
    temp_for_median
    .groupBy("DISTRICT")
    .agg(
        F.percentile_approx("crimes_each_month", 0.5, F.lit(1000000)).alias("crimes_monthly")
    ))
crime_median.show()

+--------+--------------+
|DISTRICT|crimes_monthly|
+--------+--------------+
|      C6|           532|
|      B2|          1133|
|     C11|           963|
|     E13|           401|
|      B3|           797|
|      E5|           304|
|     A15|           151|
|      A7|           318|
|     D14|           469|
|      D4|           957|
|     E18|           399|
|      A1|           778|
+--------+--------------+



● frequent_crime_types - три самых частых crime_type за всю историю наблюдений
в этом районе, объединенных через запятую с одним пробелом “, ” ,
расположенных в порядке убывания частоты
○ crime_type - первая часть NAME из таблицы offense_codes, разбитого по
разделителю “-” 


Посмотрим на таблицу codes, обратим внимание что столбец CODE отличается от OFFENSE_CODE из таблички crime, т.к. в crime они все имеют формат 5 символов, т.е. код 612 будет записан как 00612 в crime 

In [ ]:
codes.show(truncate=False)

+----+------------------------------------------+---------------------------------------+
|CODE|NAME                                      |crime_type                             |
+----+------------------------------------------+---------------------------------------+
|612 |LARCENY PURSE SNATCH - NO FORCE           |LARCENY PURSE SNATCH                   |
|613 |LARCENY SHOPLIFTING                       |LARCENY SHOPLIFTING                    |
|615 |LARCENY THEFT OF MV PARTS & ACCESSORIES   |LARCENY THEFT OF MV PARTS & ACCESSORIES|
|1731|INCEST                                    |INCEST                                 |
|3111|LICENSE PREMISE VIOLATION                 |LICENSE PREMISE VIOLATION              |
|2646|LIQUOR - DRINKING IN PUBLIC               |LIQUOR                                 |
|2204|LIQUOR LAW VIOLATION                      |LIQUOR LAW VIOLATION                   |
|3810|M/V ACCIDENT - INVOLVING �BICYCLE - INJURY|M/V ACCIDENT                           |
|3801|M/V 

Создадим новый столбец, в который запишем код в том же формате, что и в crime

In [ ]:
codes_new = codes.withColumn('zero_',
    F.when(5- F.length("CODE")==2,'00').
    otherwise('0'))

In [ ]:
codes_new =codes_new.withColumn("CODE_NEW", F.concat(F.col("zero_"), F.lit(""), (F.col("CODE")) ))
codes_new.show()

+----+--------------------+--------------------+-----+--------+
|CODE|                NAME|          crime_type|zero_|CODE_NEW|
+----+--------------------+--------------------+-----+--------+
| 612|LARCENY PURSE SNA...|LARCENY PURSE SNATCH|   00|   00612|
| 613| LARCENY SHOPLIFTING| LARCENY SHOPLIFTING|   00|   00613|
| 615|LARCENY THEFT OF ...|LARCENY THEFT OF ...|   00|   00615|
|1731|              INCEST|              INCEST|    0|   01731|
|3111|LICENSE PREMISE V...|LICENSE PREMISE V...|    0|   03111|
|2646|LIQUOR - DRINKING...|              LIQUOR|    0|   02646|
|2204|LIQUOR LAW VIOLATION|LIQUOR LAW VIOLATION|    0|   02204|
|3810|M/V ACCIDENT - IN...|        M/V ACCIDENT|    0|   03810|
|3801|M/V ACCIDENT - OTHER|        M/V ACCIDENT|    0|   03801|
|3807|M/V ACCIDENT - OT...|        M/V ACCIDENT|    0|   03807|
|3803|M/V ACCIDENT - PE...|        M/V ACCIDENT|    0|   03803|
|3805|M/V ACCIDENT - PO...|        M/V ACCIDENT|    0|   03805|
|3802|M/V ACCIDENT - PR...|        M/V A

Теперь можем заджоинить по кодам

In [ ]:
bdf = crime.join(F.broadcast(codes_new), crime.OFFENSE_CODE == codes_new.CODE_NEW)

In [ ]:
bdf.show()

+---------------+------------+--------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------------+---------+----------+--------------------+----+--------------------+--------------------+-----+--------+
|INCIDENT_NUMBER|OFFENSE_CODE|  OFFENSE_CODE_GROUP| OFFENSE_DESCRIPTION|DISTRICT|REPORTING_AREA|SHOOTING|   OCCURRED_ON_DATE|YEAR|MONTH|DAY_OF_WEEK|HOUR|  UCR_PART|           STREET|      Lat|      Long|            Location|CODE|                NAME|          crime_type|zero_|CODE_NEW|
+---------------+------------+--------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------------+---------+----------+--------------------+----+--------------------+--------------------+-----+--------+
|     I182070945|       00619|             Larceny|  LARCENY ALL OTHERS|     D14|           808|    null|2018-09-02 13:00:00|2018|    9|   

Создадим столбец, в котором проранжируем частоту типов преступления, где 1 - самый частый тип преступления

In [ ]:
frequent_crime_types_new = (
    bdf
    .groupBy("DISTRICT", 'crime_type')
    .agg(
        F.approx_count_distinct('INCIDENT_NUMBER').alias("frequent_crime_types")
    )
    .orderBy('frequent_crime_types')
    )

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
window_crime  = Window.partitionBy("DISTRICT").orderBy(F.col("frequent_crime_types").desc())

freq = frequent_crime_types_new.withColumn("row_number",row_number().over(window_crime))
freq.show()

+--------+--------------------+--------------------+----------+
|DISTRICT|          crime_type|frequent_crime_types|row_number|
+--------+--------------------+--------------------+----------+
|      A1|            PROPERTY|                2639|         1|
|      A1|SICK/INJURED/MEDICAL|                2151|         2|
|      A1|   ASSAULT & BATTERY|                2127|         3|
|      A1|      ASSAULT SIMPLE|                2127|         4|
|      A1|LARCENY THEFT FRO...|                1807|         5|
|      A1|LARCENY IN A BUIL...|                1807|         6|
|      A1|      WARRANT ARREST|                1769|         7|
|      A1|  INVESTIGATE PERSON|                1576|         8|
|      A1|INVESTIGATE PROPERTY|                1467|         9|
|      A1| TOWED MOTOR VEHICLE|                1381|        10|
|      A1|        M/V ACCIDENT|                1379|        11|
|      A1|                 M/V|                1375|        12|
|      A1|LARCENY SHOPLIFTI...|         

Создадим столбцы в которые запишем три самых частых преступления и сгруппируем по районам

In [ ]:
from pyspark.sql.functions import when

freq = freq.withColumn("crime_1", when(freq.row_number == "1",freq.crime_type)
                                 .otherwise(0))
freq = freq.withColumn("crime_2", when(freq.row_number == "2",freq.crime_type)
                                 .otherwise(0))
freq = freq.withColumn("crime_3", when(freq.row_number == "3",freq.crime_type)
                                 .otherwise(0))

In [ ]:
freq.show()

+--------+--------------------+--------------------+----------+--------+--------------------+-----------------+
|DISTRICT|          crime_type|frequent_crime_types|row_number| crime_1|             crime_2|          crime_3|
+--------+--------------------+--------------------+----------+--------+--------------------+-----------------+
|      A1|            PROPERTY|                2639|         1|PROPERTY|                   0|                0|
|      A1|SICK/INJURED/MEDICAL|                2151|         2|       0|SICK/INJURED/MEDICAL|                0|
|      A1|   ASSAULT & BATTERY|                2127|         3|       0|                   0|ASSAULT & BATTERY|
|      A1|      ASSAULT SIMPLE|                2127|         4|       0|                   0|                0|
|      A1|LARCENY THEFT FRO...|                1807|         5|       0|                   0|                0|
|      A1|LARCENY IN A BUIL...|                1807|         6|       0|                   0|           

In [ ]:
final_freq = (
    freq
    .groupBy("DISTRICT")
    .agg(
        F.max('crime_1').alias("crime_1-1"),
         F.max('crime_2').alias("crime_2-1"),
         F.max('crime_3').alias("crime_3-1")
    )
    .orderBy('DISTRICT')
    )
final_freq.show()

+--------+--------------------+--------------------+--------------------+
|DISTRICT|           crime_1-1|           crime_2-1|           crime_3-1|
+--------+--------------------+--------------------+--------------------+
|      A1|            PROPERTY|SICK/INJURED/MEDICAL|   ASSAULT & BATTERY|
|     A15|        M/V ACCIDENT|  INVESTIGATE PERSON|                 M/V|
|      A7|SICK/INJURED/MEDICAL|  INVESTIGATE PERSON|        M/V ACCIDENT|
|      B2|                 M/V|      VERBAL DISPUTE|SICK/INJURED/MEDICAL|
|      B3|      VERBAL DISPUTE|  INVESTIGATE PERSON|                 M/V|
|     C11|                 M/V|SICK/INJURED/MEDICAL|  INVESTIGATE PERSON|
|      C6|SICK/INJURED/MEDICAL|                 M/V|               DRUGS|
|     D14| TOWED MOTOR VEHICLE|                 M/V|SICK/INJURED/MEDICAL|
|      D4|LARCENY SHOPLIFTI...| LARCENY SHOPLIFTING|            PROPERTY|
|     E13|SICK/INJURED/MEDICAL|                 M/V|        M/V ACCIDENT|
|     E18|SICK/INJURED/MEDICAL|       

Склеим строки как сказано в тз, т.е. в один столбец с разделителем ","

In [ ]:
from pyspark.sql.functions import concat, lit, col
 
final_crime=final_freq.select("DISTRICT", concat(col("crime_1-1"),lit(", "), col("crime_2-1"),lit(", "), col("crime_3-1")).alias("frequent_crime_types"))
final_crime.show(truncate = False)

+--------+--------------------------------------------------------------+
|DISTRICT|frequent_crime_types                                          |
+--------+--------------------------------------------------------------+
|A1      |PROPERTY, SICK/INJURED/MEDICAL, ASSAULT & BATTERY             |
|A15     |M/V ACCIDENT, INVESTIGATE PERSON, M/V                         |
|A7      |SICK/INJURED/MEDICAL, INVESTIGATE PERSON, M/V ACCIDENT        |
|B2      |M/V, VERBAL DISPUTE, SICK/INJURED/MEDICAL                     |
|B3      |VERBAL DISPUTE, INVESTIGATE PERSON, M/V                       |
|C11     |M/V, SICK/INJURED/MEDICAL, INVESTIGATE PERSON                 |
|C6      |SICK/INJURED/MEDICAL, M/V, DRUGS                              |
|D14     |TOWED MOTOR VEHICLE, M/V, SICK/INJURED/MEDICAL                |
|D4      |LARCENY SHOPLIFTING $200 & OVER, LARCENY SHOPLIFTING, PROPERTY|
|E13     |SICK/INJURED/MEDICAL, M/V, M/V ACCIDENT                       |
|E18     |SICK/INJURED/MEDICAL, M/V, I

Склеим всю информацию в общую таблицу

In [ ]:
agregate_crime = total_crime.join(F.broadcast(final_crime), ['DISTRICT'])
agregate_crime.show(truncate=False)

+--------+------------+------------------+------------------+--------------------------------------------------------------+
|DISTRICT|crimes_total|lat               |lng               |frequent_crime_types                                          |
+--------+------------+------------------+------------------+--------------------------------------------------------------+
|C6      |20337       |42.2121224773585  |-70.85561028359758|SICK/INJURED/MEDICAL, M/V, DRUGS                              |
|B2      |41788       |42.3160037118871  |-71.07569935580469|M/V, VERBAL DISPUTE, SICK/INJURED/MEDICAL                     |
|C11     |39282       |42.29263737128488 |-71.05125984476538|M/V, SICK/INJURED/MEDICAL, INVESTIGATE PERSON                 |
|E13     |16582       |42.30980363405438 |-71.0980047618084 |SICK/INJURED/MEDICAL, M/V, M/V ACCIDENT                       |
|B3      |32197       |42.283059397131744|-71.07894938323396|VERBAL DISPUTE, INVESTIGATE PERSON, M/V                       |


In [ ]:
agregate_crime_all = agregate_crime.join(F.broadcast(crime_median), ['DISTRICT'])
agregate_crime_all.show(truncate=False)

+--------+------------+------------------+------------------+--------------------------------------------------------------+--------------+
|DISTRICT|crimes_total|lat               |lng               |frequent_crime_types                                          |crimes_monthly|
+--------+------------+------------------+------------------+--------------------------------------------------------------+--------------+
|C6      |20337       |42.2121224773585  |-70.85561028359758|SICK/INJURED/MEDICAL, M/V, DRUGS                              |532           |
|B2      |41788       |42.3160037118871  |-71.07569935580469|M/V, VERBAL DISPUTE, SICK/INJURED/MEDICAL                     |1133          |
|C11     |39282       |42.29263737128488 |-71.05125984476538|M/V, SICK/INJURED/MEDICAL, INVESTIGATE PERSON                 |963           |
|E13     |16582       |42.30980363405438 |-71.0980047618084 |SICK/INJURED/MEDICAL, M/V, M/V ACCIDENT                       |401           |
|B3      |32197     

In [ ]:
agregate_crime_all.write.save('agregate_crime_all.parquet', format='parquet')

Убедимся, что все работает, скачаем файл и откроем его

In [ ]:
codes = spark.read.csv('offense_codes.csv', header = True)

In [ ]:
# Чтение parquet файла
parquet_file = '/content/part-00000-25bcf0a2-0132-445c-a069-92d42d23bdd4-c000.snappy.parquet'
data1 = spark.read.parquet(parquet_file)

In [ ]:
data1.show()

+--------+------------+------------------+------------------+--------------------+--------------+
|DISTRICT|crimes_total|               lat|               lng|frequent_crime_types|crimes_monthly|
+--------+------------+------------------+------------------+--------------------+--------------+
|      C6|       20337|  42.2121224773585|-70.85561028359758|SICK/INJURED/MEDI...|           532|
|      B2|       41788|  42.3160037118871|-71.07569935580469|M/V, VERBAL DISPU...|          1133|
|     C11|       39282| 42.29263737128488|-71.05125984476538|M/V, SICK/INJURED...|           963|
|     E13|       16582| 42.30980363405438| -71.0980047618084|SICK/INJURED/MEDI...|           401|
|      B3|       32197|42.283059397131744|-71.07894938323396|VERBAL DISPUTE, I...|           797|
|      E5|       11669| 42.19796996617494|-71.00440886354875|SICK/INJURED/MEDI...|           304|
|     A15|        5779| 42.17915540166574|-70.74472495377354|M/V ACCIDENT, INV...|           151|
|      A7|       118